In [2]:
import pygame
import random
import numpy 

pygame.init()

# Constants
WINDOW_SIZE = (600, 600)  # Window size for 2D environment
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
PURPLE = (128, 0, 128)  # Color for the averaged dot
FONT_COLOR = (0, 0, 0)
FONT_SIZE = 24
step_size = 10
tolerance = 10  # How close the agent needs to be to a goal to trigger the end
noise_level = 15  # Noise level for second agent's movement

# Set up display
screen = pygame.display.set_mode(WINDOW_SIZE)
pygame.display.set_caption("2D Environment with Moving Target")

# Load font for rendering text
font = pygame.font.Font(None, FONT_SIZE)

# Agent 1 (controlled by the user) starts in the middle of the window
agent1_pos = [300, 300]  # Middle of the window
reached_goal1 = False  # Track if agent 1 has reached a goal

# Agent 2 (autonomous) also starts in the middle
agent2_pos = [300, 300]
reached_goal2 = False  # Track if agent 2 has reached a goal

# Agent 3 (averaged movement) also starts in the middle
agent3_pos = [300, 300]  # This will be updated by averaging agent 1 and agent 2

# Target (goal) is placed randomly in the 2D space
target_pos = [random.randint(0, WINDOW_SIZE[0]), random.randint(0, WINDOW_SIZE[1])]

# Function to move agent 1 (controlled by the user)
def move_agent1(direction):
    global agent1_pos
    # Move based on direction
    if direction == "LEFT":
        new_pos = [agent1_pos[0] - step_size, agent1_pos[1]]
    elif direction == "RIGHT":
        new_pos = [agent1_pos[0] + step_size, agent1_pos[1]]
    elif direction == "UP":
        new_pos = [agent1_pos[0], agent1_pos[1] - step_size]
    elif direction == "DOWN":
        new_pos = [agent1_pos[0], agent1_pos[1] + step_size]

    # Ensure the agent stays within the window bounds
    if 0 <= new_pos[0] <= WINDOW_SIZE[0] and 0 <= new_pos[1] <= WINDOW_SIZE[1]:
        agent1_pos = new_pos  # Update agent position if within bounds

    # Check if agent 1 has reached the target
    global reached_goal1
    reached_goal1 = (abs(agent1_pos[0] - target_pos[0]) < tolerance and
                     abs(agent1_pos[1] - target_pos[1]) < tolerance)

# Function to move agent 2 (autonomously following the target with noise)
def move_agent2():
    global agent2_pos
    # Calculate the difference between agent 2 and the target
    diff_x = target_pos[0] - agent2_pos[0]
    diff_y = target_pos[1] - agent2_pos[1]

    # Move in the direction of the target with added noise
    if diff_x > 0:
        new_x = agent2_pos[0] + step_size + random.uniform(-noise_level, noise_level)
    elif diff_x < 0:
        new_x = agent2_pos[0] - step_size + random.uniform(-noise_level, noise_level)
    else:
        new_x = agent2_pos[0]

    if diff_y > 0:
        new_y = agent2_pos[1] + step_size + random.uniform(-noise_level, noise_level)
    elif diff_y < 0:
        new_y = agent2_pos[1] - step_size + random.uniform(-noise_level, noise_level)
    else:
        new_y = agent2_pos[1]

    # Ensure the agent stays within the window bounds
    if 0 <= new_x <= WINDOW_SIZE[0]:
        agent2_pos[0] = new_x
    if 0 <= new_y <= WINDOW_SIZE[1]:
        agent2_pos[1] = new_y

    # Check if agent 2 has reached the target
    global reached_goal2
    reached_goal2 = (abs(agent2_pos[0] - target_pos[0]) < tolerance and
                     abs(agent2_pos[1] - target_pos[1]) < tolerance)

# Function to calculate the average position of agent 1 and agent 2
def update_agent3():
    global agent3_pos
    agent3_pos[0] = (agent1_pos[0] + agent2_pos[0]) / 2
    agent3_pos[1] = (agent1_pos[1] + agent2_pos[1]) / 2

# Function to reset the environment
def reset():
    global agent1_pos, agent2_pos, agent3_pos, reached_goal1, reached_goal2, target_pos
    agent1_pos = [300, 300]  # Reset agent 1 to the center
    agent2_pos = [300, 300]  # Reset agent 2 to the center
    agent3_pos = [300, 300]  # Reset agent 3 to the center
    reached_goal1 = False  # Reset goal flag for agent 1
    reached_goal2 = False  # Reset goal flag for agent 2
    target_pos = [random.randint(0, WINDOW_SIZE[0]), random.randint(0, WINDOW_SIZE[1])]  # New random target position

# Function to render the environment and GUI
def render():
    # Clear the screen
    screen.fill(WHITE)

    # Draw agent 1 (green circle) at its 2D position
    pygame.draw.circle(screen, GREEN, (int(agent1_pos[0]), int(agent1_pos[1])), 10)
    # Draw the "W" label for agent 1
    text_surface = font.render("W", True, FONT_COLOR)
    screen.blit(text_surface, (agent1_pos[0] - 5, agent1_pos[1] - 12))

    # Draw agent 2 (blue circle) at its 2D position
    pygame.draw.circle(screen, BLUE, (int(agent2_pos[0]), int(agent2_pos[1])), 10)
    # Draw the "H" label for agent 2
    text_surface = font.render("H", True, FONT_COLOR)
    screen.blit(text_surface, (agent2_pos[0] - 5, agent2_pos[1] - 12))

    # Draw agent 3 (purple circle) at its averaged position
    pygame.draw.circle(screen, PURPLE, (int(agent3_pos[0]), int(agent3_pos[1])), 10)
    # Draw the "X" label for agent 3
    text_surface = font.render("X", True, FONT_COLOR)
    screen.blit(text_surface, (agent3_pos[0] - 5, agent3_pos[1] - 12))

    # Draw the target (red circle) at its 2D position
    pygame.draw.circle(screen, RED, (int(target_pos[0]), int(target_pos[1])), 10)

    # If either agent reaches the target, display a message
    if reached_goal1 or reached_goal2:
        text = font.render("Target Reached! Press R to reset", True, FONT_COLOR)
        screen.blit(text, (150, 50))

    # Update the display
    pygame.display.update()

# Main loop
running = True
while running:
    moved = False
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_r:  # Reset the game if 'R' is pressed
                reset()

    # Check if agent 1 has reached a goal
    if not reached_goal1:  # Only allow movement if goal hasn't been reached
        # Get keys pressed
        keys = pygame.key.get_pressed()

        # Move based on keys
        if keys[pygame.K_LEFT]:      # Move left
            move_agent1("LEFT")
            moved = True
        if keys[pygame.K_RIGHT]:     # Move right
            move_agent1("RIGHT")
            moved = True
        if keys[pygame.K_UP]:        # Move up
            move_agent1("UP")
            moved = True
        if keys[pygame.K_DOWN]:      # Move down
            move_agent1("DOWN")
            moved = True

        # Only move agent 2 when agent 1 moves
        if moved and not reached_goal2:
            move_agent2()

        # Update the position of agent 3 (averaged agent)
        update_agent3()

    # Render the environment
    render()

    pygame.time.wait(100)  # Slow down for visibility

# Quit Pygame
pygame.quit()
